<a href="https://colab.research.google.com/github/HWP-Wilson/Classificador_de_tweets_com_TensorFlow_Keras_RNN/blob/main/RNN_classifica%C3%A7%C3%A3o_de_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Classificador de tweets com RNN/TensorFlow/Keras
helanowilson@ufc.br

In [ ]:
!pip install wordninja
!pip install textblob

In [ ]:
!pip install NLTK

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import json
import pickle
import urllib
import collections
from collections import Counter
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import wordninja
import textblob

from sklearn.preprocessing import MultiLabelBinarizer


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jamili\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jamili\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!pip install tensorflow

In [ ]:
!pip install tensorflow_hub 

In [ ]:
!pip install tensorflow_datasets

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
print(tf.__version__)

2.4.1


In [ ]:
import string
from nltk.corpus import stopwords
import re

In [ ]:
!pip install TextBlob

In [ ]:
#eu
dados = pd.read_csv('dados/labeled_data.csv')
dados['tweet'].head()

0    !!! RT @mayasolovely: As a woman you shouldn't...
1    !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2    !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3    !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4    !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
Name: tweet, dtype: object

In [ ]:
# Excluindo da descrição os números, informações julgadas irrelevantes para a classificação.
dados['tweet_novo'] = dados['tweet'].str.replace('[0-9]+', '', regex=True).copy()
# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
dados['tweet_novo'] = dados['tweet_novo'].str.replace('[,.:;!?]+', ' ', regex=True).copy()
# Excluindo da descrição caracteres especiais, informações julgadas irrelevantes para a classificação.
dados['tweet_novo'] = dados['tweet_novo'].str.replace("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", regex=True).copy()

# Colocando todos os caracteres em caixa baixa.
dados['tweet_novo'] = dados['tweet_novo'].str.lower().copy()
dados.head(2)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,tweet_novo
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,rt as a woman you shouldn't complain abou...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,rt boy dats cold tyga dwn bad for cuffin ...


In [ ]:
# Função para retirar stop words
punct = list(string.punctuation)
stop_words = stopwords.words('english')
additional_stop_words = ['RT', 'rt', 'via', '...', 'http', 'twitpic',
'tinyurl' ,'www']
stopword_list = punct + stop_words + additional_stop_words
def tokenize_df(tokenized_words):
  tokenized_words = word_tokenize(tokenized_words)
  stop = [word for word in tokenized_words if word not in stopword_list]
  text = TreebankWordDetokenizer().detokenize(stop)
  return text
# Eliminando as stop words
dados['tweet_organizado'] = dados['tweet_novo'].apply(tokenize_df).copy()
dados.head(2)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,tweet_novo,tweet_organizado
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,rt as a woman you shouldn't complain abou...,womann't complain cleaning house amp man alway...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,rt boy dats cold tyga dwn bad for cuffin ...,boy dats cold tyga dwn bad cuffin dat hoe st p...


In [ ]:
# Separando palavras juntas
dados['text_split'] = dados['tweet_organizado'].apply(wordninja.split)
dados['text_novo'] = dados['text_split'].apply(TreebankWordDetokenizer().detokenize)
dados.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,tweet_novo,tweet_organizado,text_split,text_novo
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,rt as a woman you shouldn't complain abou...,womann't complain cleaning house amp man alway...,"[woman, n, ', t, complain, cleaning, house, am...",woman n' t complain cleaning house amp man alw...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,rt boy dats cold tyga dwn bad for cuffin ...,boy dats cold tyga dwn bad cuffin dat hoe st p...,"[boy, dat, s, cold, tyga, dw, n, bad, cuff, in...",boy dat s cold tyga dw n bad cuff in dat hoe s...


In [ ]:
#corrigindo palavras incorretas
dados['tweet_definitivo'] = dados['text_novo'].apply(textblob.TextBlob).apply(textblob.TextBlob.correct).apply(str)
dados.head(2)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,tweet_novo,tweet_organizado,text_split,text_novo,tweet_definitivo
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,rt as a woman you shouldn't complain abou...,womann't complain cleaning house amp man alway...,"[woman, n, ', t, complain, cleaning, house, am...",woman n' t complain cleaning house amp man alw...,woman n' t complain cleaning house amp man alw...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,rt boy dats cold tyga dwn bad for cuffin ...,boy dats cold tyga dwn bad cuffin dat hoe st p...,"[boy, dat, s, cold, tyga, dw, n, bad, cuff, in...",boy dat s cold tyga dw n bad cuff in dat hoe s...,boy dat s cold tea do n bad cuff in dat he st ...


In [ ]:
# Selecionando as unicas palavras da variável tweet_definitivo
text = list(dados.tweet_definitivo)
list_words = [item for sublist in text for item in sublist]
list_words = sorted(list_words)
only_words = set(list_words)
print('Quantidade de frases:', len(text), '\n')
print('Quantidade GERAL palavras:', len(list_words), '\n')
print('Quantidade de UNICA palavras:', len(only_words), '\n')
counter = collections.Counter(list_words)

Quantidade de frases: 24783 

Quantidade GERAL palavras: 1171270 

Quantidade de UNICA palavras: 28 



In [ ]:
#EU Max tamanho dos tweet_definitivo
dados['tweet_definitivo'].apply(len).max()

154

In [ ]:
#Dicionário para coluna de classificação dos tweets
Classe = {0: 'Discurso de ódio', 1: 'Linguagem ofensiva', 2: 'Neutro'}
dados['classe']= dados['class'].map(Classe)

In [ ]:
classificacao = dados['classe']
classificacao

0                    Neutro
1        Linguagem ofensiva
2        Linguagem ofensiva
3        Linguagem ofensiva
4        Linguagem ofensiva
                ...        
24778    Linguagem ofensiva
24779                Neutro
24780    Linguagem ofensiva
24781    Linguagem ofensiva
24782                Neutro
Name: classe, Length: 24783, dtype: object

In [ ]:
dados.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,tweet_novo,text_split,text_novo,tweet_organizado,text_new_split,classe
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,womann't complain cleaning house amp man alway...,"[woman, n, ', t, complain, cleaning, house, am...",woman n' t complain cleaning house amp man alw...,woman n complain cleaning house amp man always...,"[woman, n, complain, cleaning, house, amp, man...",Neutro
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,boy dats cold tyga dwn bad cuffin dat hoe st p...,"[boy, dat, s, cold, tyga, dw, n, bad, cuff, in...",boy dat s cold tyga dw n bad cuff in dat hoe s...,boy dat cold tyga dw n bad cuff dat hoe st place,"[boy, dat, cold, tyga, dw, n, bad, cuff, dat, ...",Linguagem ofensiva
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,dawg ever fuck bitch start cry confused shit,"[daw, g, ever, fuck, bitch, start, cry, confus...",daw g ever fuck bitch start cry confused shit,daw g ever fuck bitch start cry confused shit,"[daw, g, ever, fuck, bitch, start, cry, confus...",Linguagem ofensiva
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,_g_anderson _based look like tranny,"[g, anderson, based, look, like, tr, an, ny]",g anderson based look like tr an ny,g anderson based look like tr ny,"[g, anderson, based, look, like, tr, ny]",Linguagem ofensiva
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,shit hear might true might faker bitch told ya,"[shit, hear, might, true, might, faker, bitch,...",shit hear might true might faker bitch told ya,shit hear might true might faker bitch told ya,"[shit, hear, might, true, might, faker, bitch,...",Linguagem ofensiva


In [ ]:
#Função encoder utilizando deprecated pois tfds.features não roda mais. 
encoder = tfds.deprecated.text.SubwordTextEncoder(vocab_list=only_words)
##encoder = tfds.features.text.SubwordTextEncoder(vocab_list=only_words)
print ('Vocabulary size(text_new): {}'.format(encoder.vocab_size))

Vocabulary size(text_new): 285


In [ ]:
##Exemplo do encode e decode do tweet	
sample_string = 'woman n complain cleaning house amp man always'
# Encode
encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))
#Decode
original_string = encoder.decode(encoded_string)
print('The original string: "{}"'.format(original_string))

Encoded string is [19, 20, 14, 15, 5, 61, 5, 61, 11, 20, 14, 22, 17, 15, 21, 5, 61, 11, 17, 9, 15, 5, 21, 5, 28, 61, 13, 20, 23, 6, 9, 61, 15, 14, 22, 61, 14, 15, 5, 61, 15, 17, 19, 15, 1, 6]
The original string: "woman n complain cleaning house amp man always"


In [ ]:
#Validando a string original com a string após o decode
original_string == sample_string

True

In [ ]:
# Exemplificando a relação de index ----> word
for index in encoded_string:
  print('{} ----> {}'.format(index, encoder.decode([index])))

14570 ----> woman
17935 ---->  
4573 ----> n
17935 ---->  
9495 ----> complain
17935 ---->  
17663 ----> cleaning
17935 ---->  
9420 ----> house
17935 ---->  
11671 ----> amp
17935 ---->  
14156 ----> man
17935 ---->  
12181 ----> always


In [ ]:
# Encode para as classes (labels)
label_encode = LabelEncoder()
target = label_encode.fit_transform(dados['classe'])
target

array([2, 1, 1, ..., 1, 1, 2])

### Separando os dados em treino e teste
90% para treino e 10% para teste

In [ ]:
# Definindo feature e target
x = dados['tweet_definitivo']
y = target

x_train,x_test, y_train, y_test = train_test_split(x,y, test_size=0.1)


In [ ]:
len(dados['tweet_definitivo'])

24783

In [ ]:
dados['classe'].value_counts()

Linguagem ofensiva    19190
Neutro                 4163
Discurso de ódio       1430
Name: classe, dtype: int64

In [ ]:
# Função para criação da matriz
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

In [ ]:
# Função para encode do input
def encode_input(tweet_definitivo):
  list_x = []
  for text in tweet_definitivo:
    text_encode = encoder.encode(text)
    text_encode = pad_to_size(text_encode, 256)
    list_x.append(text_encode)
  # Convertendo x em tensor
  input_encode = tf.cast(list_x, tf.float64)
  return input_encode
# Encode do x_train e x_test
x_train = encode_input(x_train)
x_test = encode_input(x_test)

In [ ]:
# Criação do modelo de RNN
inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(encoder.vocab_size, 128)(inputs)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(3, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 128)         36480     
_________________________________________________________________
bidirectional_4 (Bidirection (None, None, 128)         98816     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 128)               98816     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
Total params: 234,499
Trainable params: 234,499
Non-trainable params: 0
_____________________________________________________

In [ ]:
# Compilando modelo e configurando o processo de treinamento
epochs = 10
batch_size = 128
steps_per_epoch=10
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="rmsprop",
              metrics=['accuracy'])
# Treinando o modelo
history = model.fit(x_train, y_train, epochs=epochs,steps_per_epoch=steps_per_epoch,
                    batch_size=batch_size,
                    validation_data=(x_test, y_test))

Epoch 1/10
10/10 [==============================] - 263s 19s/step - loss: 0.8931 - accuracy: 0.6022 - val_loss: 0.6420 - val_accuracy: 0.7822
Epoch 2/10
10/10 [==============================] - 78s 8s/step - loss: 0.6787 - accuracy: 0.7817 - val_loss: 0.6353 - val_accuracy: 0.7822
Epoch 3/10
10/10 [==============================] - 82s 8s/step - loss: 0.6609 - accuracy: 0.7760 - val_loss: 0.6237 - val_accuracy: 0.7822
Epoch 4/10
10/10 [==============================] - 80s 8s/step - loss: 0.6516 - accuracy: 0.7782 - val_loss: 0.6277 - val_accuracy: 0.7822
Epoch 5/10
10/10 [==============================] - 79s 8s/step - loss: 0.6269 - accuracy: 0.7886 - val_loss: 0.5875 - val_accuracy: 0.7822
Epoch 6/10
10/10 [==============================] - 78s 8s/step - loss: 0.6542 - accuracy: 0.7646 - val_loss: 0.5990 - val_accuracy: 0.7822
Epoch 7/10
10/10 [==============================] - 80s 8s/step - loss: 0.6345 - accuracy: 0.7644 - val_loss: 0.5923 - val_accuracy: 0.7822
Epoch 8/10
10/10 [

In [ ]:
# Testando a qualidade do modelo
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

78/78 [==============================] - 21s 264ms/step - loss: 0.6751 - accuracy: 0.7273
Test Loss: 0.6750926375389099
Test Accuracy: 0.7273094058036804


In [ ]:
# Função para predição
def sample_predict(sample_pred_text):
  encoded_sample_pred_text = encoder.encode(sample_pred_text)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))
  return (predictions)

In [ ]:
sample_predict('As a woman you shouldnt complain about cleaning up your house amp as a man you should always take the trash out')

array([[0.21377325, 0.28998736, 0.49623936]], dtype=float32)

In [ ]:
# Predição do exemplo
example = 'As a woman you shouldnt complain about cleaning up your house amp as a man you should always take the trash out'
predictions = sample_predict(example)
probabilities = [np.argmax(predictions[0])]
# Retornando os labels
new_label = label_encode.inverse_transform(probabilities)
print('O exemplo "{}" foi classificado como "{}"'.format(example, new_label[0]))

O exemplo "As a woman you shouldnt complain about cleaning up your house amp as a man you should always take the trash out" foi classificado como "Neutro"


In [ ]:
dados['tweet'][24751]

'you a pussy ass nigga and I know it nigga.'

In [ ]:
sample_predict('you a pussy ass nigga and I know it nigga')

array([[0.23892337, 0.34852612, 0.41255045]], dtype=float32)

In [ ]:
example2 = ' bitch nigga miss me with it'
predictions = sample_predict(example2)
probabilities = [np.argmax(predictions[0])]
# Retornando os labels
new_label = label_encode.inverse_transform(probabilities)
print('O exemplo "{}" foi classificado como "{}"'.format(example2, new_label[0]))

O exemplo " bitch nigga miss me with it" foi classificado como "Linguagem ofensiva"


In [ ]:
example3 = 'you a pussy ass nigga and I know it nigga'
predictions = sample_predict(example3)
probabilities = [np.argmax(predictions[0])]
# Retornando os labels
new_label = label_encode.inverse_transform(probabilities)
print('O exemplo "{}" foi classificado como "{}"'.format(example3, new_label[0]))


O exemplo "you a pussy ass nigga and I know it nigga" foi classificado como "Neutro"


In [ ]:
example4 = 'playing football with friends'
predictions = sample_predict(example4)
probabilities = [np.argmax(predictions[0])]
# Retornando os labels
new_label = label_encode.inverse_transform(probabilities)
print('O exemplo "{}" foi classificado como "{}"'.format(example4, new_label[0]))

O exemplo "playing football with friends" foi classificado como "Neutro"
